In [1]:
import itertools

import numpy as np
import pandas as pd
import networkx as nx
import obnb.data
from obnb.dataset import OpenBiomedNetBench
from tqdm import tqdm

pd.options.display.max_rows = 500

In [2]:
BARFMT = "{l_bar}{bar:40}{r_bar}{bar:-40b}"
PRINT_LATEX_TABLE = False

In [3]:
networks = [
    "HumanBaseTopGlobal",
    "HuMAP",
    "STRING",
    "ConsensusPathDB",
    "FunCoup",
    "PCNet",
    "BioGRID",
    "HumanNet",
    "ComPPIHumanInt",
    "HIPPIE",
    "BioPlex",
    "HuRI",
    "OmniPath",
    "ProteomeHD",
    "SIGNOR",
]

network_stats_list = []
for network in tqdm(networks, bar_format=BARFMT):
    g = getattr(obnb.data, network)("../datasets", log_level="WARNING")
    
    num_edges = sum(map(len, g.edge_data))
    num_nodes = g.size
    density = num_edges / num_nodes / (num_nodes - 1)
    
    network_stats_list.append(
        {
            "Network": network,
            "Weighted": g.weighted,
            "Num. nodes": f"{num_nodes:,}", 
            "Num. edges": f"{num_edges:,}",
            "Density": f"{density:.6f}",
        }
    )
network_stats_df = pd.DataFrame(network_stats_list).set_index("Network")
print(network_stats_df.style.to_latex()) if PRINT_LATEX_TABLE else network_stats_df

100%|████████████████████████████████████████| 15/15 [03:05<00:00, 12.35s/it]                                                                                               


,Weighted,Num. nodes,Num. edges,Density
Network,,,,
HumanBaseTopGlobal,True,"25,689","77,807,094",0.117908
HuMAP,True,"15,433","35,052,604",0.147180
STRING,True,"18,480","11,019,492",0.032269
ConsensusPathDB,True,"17,735","10,611,416",0.033739
FunCoup,True,"17,892","10,037,478",0.031357
PCNet,False,"18,544","5,365,116",0.015603
BioGRID,False,"19,765","1,554,790",0.003980
HumanNet,True,"18,591","2,250,780",0.006513
ComPPIHumanInt,True,"17,015","699,620",0.002417


In [4]:
labels = [
    "DISEASES",
    "DisGeNET",
    "GOBP",
    "GOCC",
    "GOMF",
]

dataset_stats_list = []
for network, label in tqdm(list(itertools.product(networks, labels)), bar_format=BARFMT):
    try:
        dataset = OpenBiomedNetBench("../datasets", network, label, version="current", log_level="WARNING")
    except ValueError:
        if network == "ProteomeHD" and label == "GOMF":
            pass
    
    num_tasks = len(dataset.label.label_ids)
    task_sizes = np.array([len(dataset.label.get_labelset(i)) for i in dataset.label.label_ids])
    
    dataset_stats_list.append(
        {
            "Network": network,
            "Label": label,
            "Num. tasks": num_tasks,
            "Num. pos. avg.": f"{task_sizes.mean():.1f}",
            "Num. pos. std.": f"{task_sizes.std():.1f}",
            "Num. pos. med.": f"{np.median(task_sizes):.1f}",
        }
    )

dataset_stats_df = pd.DataFrame(dataset_stats_list).set_index(["Label", "Network"]).sort_index()
print(network_stats_df.style.to_latex()) if PRINT_LATEX_TABLE else dataset_stats_df

100%|████████████████████████████████████████| 75/75 [26:36<00:00, 21.28s/it]                                                                                               


Num. tasks Num. pos. avg. Num. pos. std.  \
Label    Network                                                        
DISEASES BioGRID                    145          178.1          137.4   
         BioPlex                     72          123.8           64.4   
         ComPPIHumanInt             145          174.6          134.5   
         ConsensusPathDB            144          177.4          137.5   
         FunCoup                    145          177.1          135.1   
         HIPPIE                     143          178.1          137.6   
         HuMAP                      123          168.0          119.2   
         HuRI                        50          130.3           56.7   
         HumanBaseTopGlobal         149          178.5          137.7   
         HumanNet                   142          179.0          136.9   
         OmniPath                   135          180.2          131.1   
         PCNet                      143          171.8          130.6   
         ProteomeHD                  15           76.9           22.4   
         SIGNOR                      89          144.6           89.4   
         STRING                     146          175.4          135.6   
DisGeNET BioGRID                    305          208.3          143.1   
         BioPlex                    189          138.6           71.4   
         ComPPIHumanInt             301          204.1          138.7   
         ConsensusPathDB            298          207.4          140.8   
         FunCoup                    299          204.7          139.4   
         HIPPIE                     306          208.1          142.9   
         HuMAP                      279          194.3          126.7   
         HuRI                       152          122.9           54.7   
         HumanBaseTopGlobal         287          219.7          145.7   
         HumanNet                   302          204.2          140.3   
         OmniPath                   298          199.6          136.0   
         PCNet                      292          202.1          135.5   
         ProteomeHD                  56           78.0           24.8   
         SIGNOR                     219          147.3           81.9   
         STRING                     296          208.0          140.6   
GOBP     BioGRID                    114           89.5           37.1   
         BioPlex                     38           77.6           22.6   
         ComPPIHumanInt             104           91.8           37.0   
         ConsensusPathDB            112           90.1           37.0   
         FunCoup                    114           87.8           36.7   
         HIPPIE                     111           89.2           37.1   
         HuMAP                       96           84.6           32.3   
         HuRI                        27           69.9           16.0   
         HumanBaseTopGlobal         115           89.2           37.3   
         HumanNet                   117           88.6           36.9   
         OmniPath                   106           88.7           36.2   
         PCNet                      105           89.0           36.0   
         ProteomeHD                   5           80.4           22.6   
         SIGNOR                      41           81.3           22.7   
         STRING                     116           88.9           36.6   
GOCC     BioGRID                     71           96.0           36.3   
         BioPlex                     35           77.0           20.9   
         ComPPIHumanInt              68           96.1           35.5   
         ConsensusPathDB             71           95.5           35.8   
         FunCoup                     71           95.9           36.4   
         HIPPIE                      69           97.3           35.8   
         HuMAP                       62           91.8           33.4   
         HuRI                        21           69.0           12.4   
         Human